In [5]:
from huggingface_hub import snapshot_download, login
import os

login('')

os.makedirs("PhysicalAI-Spatial-Intelligence-Warehouse", exist_ok=True)

snapshot_download(
    repo_id="nvidia/PhysicalAI-Spatial-Intelligence-Warehouse",
    repo_type="dataset",
    local_dir="PhysicalAI-Spatial-Intelligence-Warehouse",
    allow_patterns="train_sample/*"
)

Fetching 201 files: 100%|██████████| 201/201 [00:00<00:00, 3195.05it/s]


'/Users/dangminh/Desktop/SpatialRGPT/PhysicalAI-Spatial-Intelligence-Warehouse'

In [4]:
!cp PhysicalAI-Spatial-Intelligence-Warehouse/train_sample/train_sample.json PhysicalAI-Spatial-Intelligence-Warehouse

In [ ]:
from llava.mm_utils import get_model_name_from_path
from llava.model.builder import load_pretrained_model # To get tokenizer and image_processor

args_model_path = "a8cheng/SpatialRGPT-VILA1.5-8B"
args_model_base = None
args_conv_mode = "llava_v1"

model_name_inferred = get_model_name_from_path(args_model_path)
print(f"model_name_inferred: {model_name_inferred}")

tokenizer, model, image_processor, context_len = load_pretrained_model(
    args_model_path,
    model_name_inferred,
    args_model_base,
    load_8bit=False,
    load_4bit=False
)
print("SpatialRGPT model loaded Successfully!")

In [3]:
# download pretrain mdoel
!git lfs install
!huggingface-cli download a8cheng/SpatialRGPT-VILA1.5-8B \
    --local-dir checkpoint/SpatialRGPT-VILA1.5-8B \
    --local-dir-use-symlinks False

Updated Git hooks.
Git LFS initialized.
/Users/dangminh/miniconda3/envs/spatialrgpt/lib/python3.10/site-packages/huggingface_hub/commands/download.py:139: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Fetching 20 files:   0%|                                 | 0/20 [00:00<?, ?it/s]Downloading 'llm/model-00004-of-00004.safetensors' to 'checkpoint/SpatialRGPT-VILA1.5-8B/.cache/huggingface/download/llm/-dFtyT7kcgbTHt1cy9JKqruJCR4=.04cd6d849f00ae7c2f88754c031bbb909035caed6571f9a194794a3c17573a21.incomplete'

model-00004-of-00004.safetensors:   0%|             | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|             | 0.00/5.00G [00:00<?, ?B/s]


model-00003-of-00004.safetensors:   0%|             | 0.00/4.92G [00:00<?, ?B/s]



generation_config.json: 100%|██████████████████| 194/194 [00:00<00:00, 1.54MB/s]
Download complete. Moving file to checkpoint/SpatialRGPT-VILA1.5-8B/llm/gene